In [1]:
import datetime
import json
import numpy as np
import time
from pathlib import Path
import pandas as pd
import torch
import torch.backends.cudnn as cudnn
# from torch.utils.tensorboard import SummaryWriter
from datetime import date
import timm
# assert timm.__version__ == "0.3.2" # version check
from copy import deepcopy

import os
import sys
sys.path.insert(1, '/mnt/c/Users/PCM/Documents/GitHub/SSL-FL/code')
sys.path.insert(1, '/mnt/c/Users/PCM/Documents/GitHub/SSL-FL/segmenter')
import fed_mae.models_vit as models_vit
from fed_mae.engine_for_finetuning import train_one_epoch
import util.misc as misc

/home/hongn/miniconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision import datasets
from torchvision import  transforms
mean = [0.6821, 0.4575, 0.2626]
std  = [0.1324, 0.1306, 0.1022]

data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

data_dir = '/mnt/c/Users/PCM/Dropbox/ee638/Diabetic'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val',  'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device, class_names)

cuda:0 ['0', '1', '2', '3', '4']


In [27]:
segm = models_vit.__dict__['vit_base_patch16'](
        num_classes=5,
        drop_path_rate=0.1,
        global_pool=True,
        )
state_dict = torch.load('/mnt/c/Users/PCM/Documents/GitHub/SSL-FL/pretrained/bestacc-severity-cls-scratch-sslfl-2024-04-25.pt')
segm.load_state_dict(state_dict, strict=True)


<All keys matched successfully>

In [28]:
valaccmax = 0
segm = segm.to(device)
training_acc = 0
val_acc = 0
training_loss_test = 0.0

predlist = []
labelist = []
problist = []
for inputs, labels in dataloaders['test']:
    segm.eval()
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = segm(inputs)
        _, preds = torch.max(outputs, 1)
        # loss = loss_fn(outputs, labels)
        problist.append(outputs[:,1].detach().cpu().numpy())
        labelist.append(labels.detach().cpu().numpy()*1)
        predlist.append(preds.detach().cpu().numpy())

    labelist = np.concatenate(labelist).ravel()
    problist = np.concatenate(problist).ravel()
    predlist = np.concatenate(predlist).ravel()
    val_acc += torch.sum(preds == labels.data)
        val_f1 = f1_score(labelist, predlist, average='macro')
if(e > 5 and val_acc >= valaccmax):
    valaccmax = val_acc
    torch.save(clf_model.state_dict(), f'./pretrained/best-mutilclass-siamese50-{pretrain_mode}.pt')
    torch.save(clf_model.state_dict(), f'./pretrained/{pretrain_mode}/bestacc-{pretrain_mode}-{epo}.pt')
if(e > 5 and sedis <= valaccmin):
    torch.save(clf_model.state_dict(), f'./pretrained/{pretrain_mode}/bestmaee-{pretrain_mode}-{epo}.pt')
if(e > 5 and val_f1 >= valf1max):
    torch.save(clf_model.state_dict(), f'./pretrained/{pretrain_mode}/bestf1-{pretrain_mode}-{epo}.pt')
scheduler.step()

In [31]:
val_acc / dataset_sizes['test']

tensor(0.7347, device='cuda:0')

: 